In [1]:
import cvxpy as cvx
import numpy as np
import timeit
from ios import *

In [2]:
bigram = get_bigram('data/processed/bigram.npy')
plainseq = get_plaintext('408plaincleaned')
plainseq = tointseq(plainseq)
plainmat = np.reshape(plainseq, (24, 17))

def toCharMat(intMat, tour=None):
    res = np.chararray(intMat.shape, unicode=True)
    if tour is None:
        for i in range(intMat.shape[1]):
            res[:,i] = tocharseq(intMat[:,i])
    else:
        for i,j in enumerate(tour):
            res[:,i] = tocharseq(intMat[:,j])
    return res   

charmat = toCharMat(plainmat)
print(charmat)

[['I' 'L' 'I' 'K' 'E' 'K' 'I' 'L' 'L' 'I' 'N' 'G' 'P' 'E' 'O' 'P' 'L']
 ['E' 'B' 'E' 'C' 'A' 'U' 'S' 'E' 'I' 'T' 'I' 'S' 'S' 'O' 'M' 'U' 'C']
 ['H' 'F' 'U' 'N' 'I' 'T' 'I' 'S' 'M' 'O' 'R' 'E' 'F' 'U' 'N' 'T' 'H']
 ['A' 'N' 'K' 'I' 'L' 'L' 'I' 'N' 'G' 'W' 'I' 'L' 'D' 'G' 'A' 'M' 'E']
 ['I' 'N' 'T' 'H' 'E' 'F' 'O' 'R' 'R' 'E' 'S' 'T' 'B' 'E' 'C' 'A' 'U']
 ['S' 'E' 'M' 'A' 'N' 'I' 'S' 'T' 'H' 'E' 'M' 'O' 'S' 'T' 'D' 'A' 'N']
 ['G' 'E' 'R' 'O' 'U' 'S' 'A' 'N' 'I' 'M' 'A' 'L' 'O' 'F' 'A' 'L' 'L']
 ['T' 'O' 'K' 'I' 'L' 'L' 'S' 'O' 'M' 'E' 'T' 'H' 'I' 'N' 'G' 'G' 'I']
 ['V' 'E' 'S' 'M' 'E' 'T' 'H' 'E' 'M' 'O' 'S' 'T' 'T' 'H' 'R' 'I' 'L']
 ['L' 'I' 'N' 'G' 'E' 'X' 'P' 'E' 'R' 'E' 'N' 'C' 'E' 'I' 'T' 'I' 'S']
 ['E' 'V' 'E' 'N' 'B' 'E' 'T' 'T' 'E' 'R' 'T' 'H' 'A' 'N' 'G' 'E' 'T']
 ['T' 'I' 'N' 'G' 'Y' 'O' 'U' 'R' 'R' 'O' 'C' 'K' 'S' 'O' 'F' 'F' 'W']
 ['I' 'T' 'H' 'A' 'G' 'I' 'R' 'L' 'T' 'H' 'E' 'B' 'E' 'S' 'T' 'P' 'A']
 ['R' 'T' 'O' 'F' 'I' 'T' 'I' 'S' 'T' 'H' 'A' 'T' 'W' 'H' 'E' 'N' 'I']
 ['D' 

In [3]:
# column permutation and revsersible mapping

perm = np.random.permutation(17)
perm_plainmat = plainmat[:,perm]
# print(toCharMat(perm_plainmat, perm=perm))

mapback = dict()
for i,j in enumerate(perm):
    mapback[i] = j

In [4]:
def col2col(a,b):
    logp = 0
    for i,j in zip(a,b):
        logp += np.log(bigram[i,j])
    return logp

def colscore(mat):
    score = 0
    for i in range(mat.shape[1] - 1):
        score += col2col(mat[:,i], mat[:,i+1])
    return score

In [5]:
# gold key score
colscore(plainmat)

-991.3825477818259

In [6]:
# construct score matrix for the permuted character matrix

A = np.zeros((18, 18))
for i in range(18):
    for j in range(18):
        if i == 17 or j == 17:
            A[i,j] = 0
        else:
            A[i,j] = col2col(perm_plainmat[:,i], perm_plainmat[:,j])
        
# AA = np.zeros([18, 18], dtype=float)
# AA[1:,1:] = A
# A = AA
# n = 18

n = 18

In [18]:
def subtour(B):
    """
    return subtour from a boolean matrix B
    """
    node = 0
    subt = [node]
    while True:
        for j in range(n):
            #print (B[subt[-1], j])
            if B[j,subt[-1]] > 0.99:
                if j not in subt:
                    subt.append(j)
                else:
                    return subt

In [19]:
"""
MTZ subtour elimination constraint
"""

# boolean matrix, indicating the trip
B = cvx.Bool(n,n)

# exemplary matrix
C = np.ones((1,n), dtype=int)

# auxiliary var
# u = cvx.Variable(n, integer=True)
u = cvx.Int(n)

# objective
obj = cvx.Maximize(sum([A[i,:]*B[:,i] for i in range(n)]))


# basic condition
constraints = [(cvx.sum_entries(B, axis=0) == C), (cvx.sum_entries(B, axis=1) == C.transpose())]

# subtour elimination
for i in range(1,n):
    for j in range(1,n):
        if i != j:
            constraints.append(u[i] - u[j] + n*B[i,j] <= n - 1)
            
# condition for u
for i in range(1,n):
    constraints.append(u[i] >= 0)
    constraints.append(u[i] <= n - 1)
    
st = timeit.default_timer()
prob = cvx.Problem(obj, constraints)

# Time performance:

opt = prob.solve(solver=cvx.GLPK_MI)

# Print results
print ("Minimal time: ", opt)
print ("Optimal tour: ", subtour(B.value))
print ("Converge time: ", timeit.default_timer() - st)


Minimal time:  -991.3825477818259
Optimal tour:  [0, 17, 12, 3, 6, 9, 16, 11, 14, 8, 4, 2, 10, 13, 1, 5, 15, 7]
Converge time:  0.3813633879981353


In [20]:
# re-map and print
tour = subtour(B.value)
dummy_node_idx = tour.index(17)
tour = tour[dummy_node_idx+1:] + tour[:dummy_node_idx]
print(tour)
proposal = perm_plainmat[:,tour]
print(proposal)
colscore(proposal)
B = B.value
print(sum([A[i,:]*B[:,i] for i in range(n)]))
print(B)

[12, 3, 6, 9, 16, 11, 14, 8, 4, 2, 10, 13, 1, 5, 15, 7, 0]
[[ 8 11  8 10  4 10  8 11 11  8 13  6 15  4 14 15 11]
 [ 4  1  4  2  0 20 18  4  8 19  8 18 18 14 12 20  2]
 [ 7  5 20 13  8 19  8 18 12 14 17  4  5 20 13 19  7]
 [ 0 13 10  8 11 11  8 13  6 22  8 11  3  6  0 12  4]
 [ 8 13 19  7  4  5 14 17 17  4 18 19  1  4  2  0 20]
 [18  4 12  0 13  8 18 19  7  4 12 14 18 19  3  0 13]
 [ 6  4 17 14 20 18  0 13  8 12  0 11 14  5  0 11 11]
 [19 14 10  8 11 11 18 14 12  4 19  7  8 13  6  6  8]
 [21  4 18 12  4 19  7  4 12 14 18 19 19  7 17  8 11]
 [11  8 13  6  4 23 15  4 17  4 13  2  4  8 19  8 18]
 [ 4 21  4 13  1  4 19 19  4 17 19  7  0 13  6  4 19]
 [19  8 13  6 24 14 20 17 17 14  2 10 18 14  5  5 22]
 [ 8 19  7  0  6  8 17 11 19  7  4  1  4 18 19 15  0]
 [17 19 14  5  8 19  8 18 19  7  0 19 22  7  4 13  8]
 [ 3  8  4  8 22  8 11 11  1  4 17  4  1 14 17 13  8]
 [13 15  0 17  0  3  8  2  4  0 13  3  0 11 11 19  7]
 [ 4  8  7  0 21  4 10  8 11 11  4  3 22  8 11 11  1]
 [ 4  2 14 12  4 12 24 

In [22]:
"""
Lazy subtour elimination
"""

# boolean matrix, indicating the trip
B = cvx.Bool(n,n)

# exemplary matrix
C = np.ones((1,n), dtype=int)

# objective
obj = cvx.Maximize(sum([A[i,:]*B[:,i] for i in range(n)]))

# basic condition
constraints = [(cvx.sum_entries(B, axis=0) == C), (cvx.sum_entries(B, axis=1) == C.transpose())]

# preliminary solution, which might involve subtours
prob = cvx.Problem(obj, constraints)
st = timeit.default_timer()
opt = prob.solve()

# while True:
#     subt = subtour(B.value)
#     if len(subt) == n:
#         print ("Minimal time: ", opt)
#         print ("Optimal tour: ", subt)
#         print ("Converge time: ", timeit.default_timer() - st)
#         break
#     else:
#         print ("Try: ", subt)
#         nots = [j for j in range(n) if j not in subt]
#         constraints.append(sum(B[i,j] for i in subt for j in nots) >= 1)
#         prob = cvx.Problem(obj, constraints)
#         opt = prob.solve(solver=cvx.GLPK_MI)

In [23]:
B.value

matrix([[4.57241742e-12, 9.93238988e-12, 7.21044594e-12, 6.40445233e-12,
         8.18757096e-12, 8.67529711e-12, 6.12914762e-12, 1.00000000e+00,
         1.00407061e-11, 5.80186073e-12, 5.50041079e-12, 9.72292235e-12,
         6.88746064e-12, 2.45909686e-11, 6.69580452e-12, 6.38619535e-12,
         7.04627926e-12, 3.10228915e-11],
        [5.92451961e-12, 6.97119651e-12, 9.01792539e-12, 8.54939731e-12,
         7.08193903e-12, 1.47544553e-11, 9.97142297e-12, 1.58688535e-11,
         2.05064799e-11, 7.22091006e-12, 6.65253731e-12, 8.43105009e-12,
         1.29080317e-11, 1.00000000e+00, 1.09544003e-11, 9.59850349e-12,
         7.93450881e-12, 2.71640931e-10],
        [6.34640459e-12, 9.05500112e-12, 3.51088182e-12, 6.78683341e-12,
         1.00000000e+00, 5.70868701e-12, 4.81060660e-12, 6.46590322e-12,
         9.11521981e-12, 6.90755046e-12, 6.45806835e-12, 1.02107433e-11,
         7.17613479e-12, 1.07709692e-11, 4.83834320e-12, 1.44058511e-11,
         5.52201745e-12, 7.36819792e-12]